In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Download Ridership Data from Toronto Open Data

Instructions for the code to download the data is obtained from the Toronto Open Data website (https://open.toronto.ca/dataset/bike-share-toronto-ridership-data/) and modified for use.

In [ ]:
# Download relevant ridership data from source (only data from 2019 to 2024)

import requests

# Set parameters
base_url = "https://ckan0.cf.opendata.inter.prod-toronto.ca"
url = base_url + "/api/3/action/package_show"
params = { "id": "bike-share-toronto-ridership-data"}
package = requests.get(url, params = params).json()

# To get resource data:
for idx, resource in enumerate(package["result"]["resources"]):

      # To get metadata for non datastore_active resources:
      if not resource["datastore_active"]:
          url = base_url + "/api/3/action/resource_show?id=" + resource["id"]
          resource_metadata = requests.get(url).json()
          print(resource_metadata)

          # 2019 to 2023 not found in resource["name"] then skip
          if "2019" not in resource["name"] and \
            "2020" not in resource["name"] and \
            "2021" not in resource["name"] and \
            "2022" not in resource["name"] and \
            "2023" not in resource["name"] and \
            "2024" not in resource["name"]:
            continue

          # Download data from url
          r = requests.get(resource["url"])
          with open(resource["name"], 'wb') as f:
              f.write(r.content)

{'help': 'https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/help_show?name=resource_show', 'success': True, 'result': {'cache_last_updated': None, 'cache_url': None, 'created': '2019-07-23T16:40:47.062116', 'datastore_active': False, 'format': 'XLSX', 'hash': '', 'id': 'ad78f9f7-d4f2-42a1-9a1c-61dbc83f84c8', 'is_datastore_cache_file': False, 'is_preview': 'False', 'last_modified': '2022-04-12T14:38:55', 'metadata_modified': '2022-04-12T18:30:10.592092', 'mimetype': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet', 'mimetype_inner': None, 'name': 'bikeshare-ridership-readme', 'package_id': '7e876c24-177c-4605-9cef-e50dd74c617f', 'position': 0, 'resource_type': None, 'revision_id': '890aa1da-c8d0-4535-a719-14786ce56443', 'size': 16625, 'state': 'active', 'url': 'https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/7e876c24-177c-4605-9cef-e50dd74c617f/resource/ad78f9f7-d4f2-42a1-9a1c-61dbc83f84c8/download/bikeshare-ridership-readme.xlsx', 'url_type': 'uplo

In [ ]:
# Unzip ridership data downloaded

import zipfile
import os

years = [2019, 2020, 2021, 2022, 2023, 2024]
base_path = '/content'

for year in years:
    year_dir = os.path.join(base_path, str(year))
    os.makedirs(year_dir, exist_ok=True)

    zip_file = f'/content/bikeshare-ridership-{year}'

    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(year_dir)

In [ ]:
# Some unzipped data files have folder within folders, hence necessary to shift contents to parent directory for that year

import os
import shutil

def shift_contents_to_parent_directory(subdirectory_path):
    parent_directory = os.path.dirname(subdirectory_path)

    for item in os.listdir(subdirectory_path):
        item_path = os.path.join(subdirectory_path, item)
        destination_path = os.path.join(parent_directory, item)

        shutil.move(item_path, destination_path)

    os.rmdir(subdirectory_path)

years = [2022, 2023]

for year in years:
    try:
        subdirectory_path = f'/content/{year}/bikeshare-ridership-{year}'
        shift_contents_to_parent_directory(subdirectory_path)
    except FileNotFoundError:
        print(f"Subdirectory '{subdirectory_path}' not found.")

In [ ]:
import os
import pandas as pd

def combine_csv_files(directory_path):
    # List to hold individual DataFrames
    dataframes = []

    # Loop through all files in the directory
    for file in os.listdir(directory_path):
        if file.endswith('.csv'):
            # Read each CSV file into a DataFrame
            try:
                df = pd.read_csv(os.path.join(directory_path, file), encoding='utf-8')
            except UnicodeDecodeError:
                df = pd.read_csv(os.path.join(directory_path, file), encoding='latin1')
            dataframes.append(df)

    # Concatenate all DataFrames into one
    combined_df = pd.concat(dataframes, ignore_index=True)

    return combined_df

years = [2019, 2021, 2022, 2023, 2024]

for year in years:
    try:
        directory_path = f'/content/{year}'  # Change to your directory path
        combined_df = combine_csv_files(directory_path)

        # To save the combined DataFrame to a CSV file, if needed
        combined_df.to_csv(os.path.join(directory_path, f'combined-{year}.csv'), index=False)
    except:
        print(f"Error processing year {year}")

Error processing year 2021
Error processing year 2022
Error processing year 2023
Error processing year 2024


In [ ]:
# Combine data from all years (2019 to 2023; 2024 will be used later as final test dataset) into one combined csv file for later use

import os
import pandas as pd

def combine_csv_files_from_directories(base_directory, years):
    dataframes = []

    for year in years:
        directory_path = os.path.join(base_directory, str(year))

        for file in os.listdir(directory_path):
            if file.startswith('combined'):
                try:
                    df = pd.read_csv(os.path.join(directory_path, file), encoding='utf-8')
                except UnicodeDecodeError:
                    df = pd.read_csv(os.path.join(directory_path, file), encoding='latin1')
                dataframes.append(df)

    combined_df = pd.concat(dataframes, ignore_index=True)
    return combined_df

base_directory = '/content'
years = [2019, 2020, 2021, 2022, 2023]
combined_ridership_df = combine_csv_files_from_directories(base_directory, years)

In [ ]:
# Clean data by dropping rows with NA values and saving as csv
combined_ridership_df.dropna(inplace=True)
combined_ridership_df.to_csv(os.path.join(base_directory, 'combined_all_years_cleaned.csv'), index=False)

In [ ]:
# Clean and save 2024 ridership data
ridership_2024_df = pd.read_csv('/content/2024/Bike share ridership 2024-01.csv')
ridership_2024_df.dropna(inplace=True)
ridership_2024_df.to_csv(os.path.join('/content/', 'ridership_2024_cleaned.csv'), index=False)

# Download Weather Data from Government of Canada

Instructions for the code to download the data is obtained from the Government of Canada website (https://collaboration.cmc.ec.gc.ca/cmc/climate/Get_More_Data_Plus_de_donnees/) and modified for use.

In [ ]:
# Download relevant weather data from source (only data from 2019 to 2024)

import os
import requests

base_url = "https://climate.weather.gc.ca/climate_data/bulk_data_e.html"
params = {
    "format": "csv",
    "stationID": 51459, # For Toronto
    "Day": 14,
    "timeframe": 1,
    "submit": "Download Data"
}

os.makedirs("climate_data", exist_ok=True)

for year in range(2019, 2025):
    for month in range(1, 13):
        params["Year"] = year
        params["Month"] = month

        response = requests.get(base_url, params=params)

        if response.status_code == 200:
            content_disposition = response.headers.get("content-disposition")
            if content_disposition:
                filename = content_disposition.split("filename=")[-1].strip('"')
            else:
                filename = f"climate_data_{year}_{month}.csv"

            file_path = os.path.join("climate_data", filename)
            with open(file_path, "wb") as file:
                file.write(response.content)
            print(f"Downloaded: {filename}")
        else:
            print(f"Failed to download data for {year}-{month}")

Downloaded: en_climate_hourly_ON_6158731_01-2019_P1H.csv
Downloaded: en_climate_hourly_ON_6158731_02-2019_P1H.csv
Downloaded: en_climate_hourly_ON_6158731_03-2019_P1H.csv
Downloaded: en_climate_hourly_ON_6158731_04-2019_P1H.csv
Downloaded: en_climate_hourly_ON_6158731_05-2019_P1H.csv
Downloaded: en_climate_hourly_ON_6158731_06-2019_P1H.csv
Downloaded: en_climate_hourly_ON_6158731_07-2019_P1H.csv
Downloaded: en_climate_hourly_ON_6158731_08-2019_P1H.csv
Downloaded: en_climate_hourly_ON_6158731_09-2019_P1H.csv
Downloaded: en_climate_hourly_ON_6158731_10-2019_P1H.csv
Downloaded: en_climate_hourly_ON_6158731_11-2019_P1H.csv
Downloaded: en_climate_hourly_ON_6158731_12-2019_P1H.csv
Downloaded: en_climate_hourly_ON_6158731_01-2020_P1H.csv
Downloaded: en_climate_hourly_ON_6158731_02-2020_P1H.csv
Downloaded: en_climate_hourly_ON_6158731_03-2020_P1H.csv
Downloaded: en_climate_hourly_ON_6158731_04-2020_P1H.csv
Downloaded: en_climate_hourly_ON_6158731_05-2020_P1H.csv
Downloaded: en_climate_hourly_O

In [ ]:
import os
import pandas as pd

def combine_csv_files(base_directory):
    dataframes = []

    for file in os.listdir(base_directory):
        if file.endswith('.csv'):
            try:
                df = pd.read_csv(os.path.join(base_directory, file), encoding='utf-8')
            except UnicodeDecodeError:
                df = pd.read_csv(os.path.join(base_directory, file), encoding='latin1')
            dataframes.append(df)

    combined_df = pd.concat(dataframes, ignore_index=True)
    return combined_df

base_directory = '/content/climate_data'
combined_weather_df = combine_csv_files(base_directory)

combined_weather_df.to_csv(os.path.join(base_directory, 'weather_all_years.csv'), index=False)

In [ ]:
# Drop unnecessary columns, and "Precip. Amount (mm)" because all values are NA

combined_weather_df.drop(['Longitude (x)', 'Latitude (y)', 'Station Name', 'Climate ID',
                  'Temp Flag', 'Dew Point Temp Flag', 'Rel Hum Flag',
                  'Precip. Amount Flag', 'Wind Dir Flag', 'Wind Spd Flag',
                  'Visibility Flag', 'Stn Press Flag', 'Hmdx Flag', 'Wind Chill Flag'
                 ], axis=1, inplace=True)

combined_weather_df.drop('Precip. Amount (mm)', axis=1, inplace=True)

In [ ]:
# Count number of NaN per column
nan_counts = combined_weather_df.isna().sum()

# Print the results
print(nan_counts)
print(len(combined_weather_df))

Date/Time (LST)            0
Year                       0
Month                      0
Day                        0
Time (LST)                 0
Temp (°C)               3486
Dew Point Temp (°C)     3486
Rel Hum (%)             3486
Wind Dir (10s deg)      3734
Wind Spd (km/h)         3485
Visibility (km)         3486
Stn Press (kPa)         3485
Hmdx                   44099
Wind Chill             42703
Weather                31385
dtype: int64
52608


In [ ]:
# Drop "Hmdx" and "Wind Chill" because too many NA (more than half)
# Keep "Weather" for now (though it also has many NA) - will do a .fillna() later
combined_weather_df.drop(['Hmdx', 'Wind Chill'], axis=1, inplace=True)

In [ ]:
# Sort by "Date/Time (LST)" to do fillna()
combined_weather_df.sort_values('Date/Time (LST)', inplace=True)

# Fill "Weather" using fillna()
combined_weather_df["Weather"].fillna(method='ffill', inplace=True)

<ipython-input-12-0339d9d30f66>:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_weather_df["Weather"].fillna(method='ffill', inplace=True)


In [ ]:
# Drop NA from "Temp (°C)", "Dew Point Temp (°C)", "Wind Dir (10s deg)", "Wind Spd (km/h)",
# "Visibility (km)" because NA do not represent 0 since 0 values were already present in the
# data for those columns
# Drop NA from "Stn Press (kPa)" because atmosphere pressure cannot be 0
# Drop NA from "Rel Hum (%)" because only 30 data points of NA
# (and most of the data points with NA for "Rel Hum (%)" are also NA for most other columns)

combined_weather_df.dropna(subset=['Temp (°C)',
                           'Dew Point Temp (°C)',
                           'Wind Dir (10s deg)',
                           'Wind Spd (km/h)',
                           'Visibility (km)',
                           'Stn Press (kPa)',
                           'Rel Hum (%)'], inplace=True)

In [ ]:
# Save cleaned weather data as csv
combined_weather_df.to_csv('/content/drive/MyDrive/Research - Summer 2024/weather_all_years_cleaned.csv', index=False)

# Data Preprocessing

1. Aggregate ridership data by check-ins and check-outs per hour
2. Combine aggregated ridership data with weather data
3. Binarise "Weather" column of combined data into respective categories
4. Aggregate binarised weather categories into more general categories to reduce dimension of data

**1. Aggregate ridership data by check-ins and check-outs per hour**

In [ ]:
import pandas as pd
ridership_df = pd.read_csv('/content/combined_all_years_cleaned.csv')

In [ ]:
# Convert "Start Time" and "End Time" to datetime object
ridership_df['Start Time'] = pd.to_datetime(ridership_df['Start Time'])
ridership_df['End Time'] = pd.to_datetime(ridership_df['End Time'])

# Extract hour and date features from 'Start Time' and 'End Time'
ridership_df['Start Hour'] = ridership_df['Start Time'].dt.floor('H')
ridership_df['End Hour'] = ridership_df['End Time'].dt.floor('H')

In [ ]:
# Aggregate check-outs per hour per station
check_outs = ridership_df.groupby(['Start Station Id', 'Start Hour']).size().reset_index(name='Check-Outs')

# Aggregate check-ins per hour per station
check_ins = ridership_df.groupby(['End Station Id', 'End Hour']).size().reset_index(name='Check-Ins')

# Rename columns for clarity
check_outs.rename(columns={'Start Station Id': 'Station Id', 'Start Hour': 'Hour'}, inplace=True)
check_ins.rename(columns={'End Station Id': 'Station Id', 'End Hour': 'Hour'}, inplace=True)

# Merge check-outs and check-ins dataframes
aggregated_data = pd.merge(check_outs, check_ins, on=['Station Id', 'Hour'], how='outer').fillna(0)

# Sort by Station Id and Hour for better readability
aggregated_data.sort_values(by=['Station Id', 'Hour'], inplace=True)

**2. Combine aggregated ridership data with weather data**

In [ ]:
weather_df = pd.read_csv('/content/drive/MyDrive/Research - Summer 2024/weather_all_years_cleaned.csv')

In [ ]:
# Convert 'Date/Time (LST)' column to datetime
weather_df['Date/Time (LST)'] = pd.to_datetime(weather_df['Date/Time (LST)'])

# Extract the hour feature from 'time'
weather_df['Hour'] = weather_df['Date/Time (LST)'].dt.floor('H')
weather_df.head()

,Date/Time (LST),Year,Month,Day,Time (LST),Temp (°C),Dew Point Temp (°C),Rel Hum (%),Wind Dir (10s deg),Wind Spd (km/h),Visibility (km),Stn Press (kPa),Weather,Hour
0,2019-01-01 00:00:00,2019,1,1,00:00,4.5,3.7,94.0,18.0,14.0,6.4,97.29,"Drizzle,Fog",2019-01-01 00:00:00
1,2019-01-01 01:00:00,2019,1,1,01:00,4.4,3.6,94.0,27.0,7.0,9.7,97.32,"Drizzle,Fog",2019-01-01 01:00:00
2,2019-01-01 02:00:00,2019,1,1,02:00,2.4,1.5,94.0,33.0,35.0,9.7,97.54,"Rain,Drizzle,Fog",2019-01-01 02:00:00
3,2019-01-01 03:00:00,2019,1,1,03:00,2.2,1.2,93.0,31.0,29.0,19.3,97.89,"Rain,Drizzle,Fog",2019-01-01 03:00:00
4,2019-01-01 04:00:00,2019,1,1,04:00,1.9,0.8,92.0,31.0,30.0,24.1,98.16,Cloudy,2019-01-01 04:00:00


In [ ]:
# Merge the weather data with the ridership data
merged_df = pd.merge(aggregated_data, weather_df, on='Hour', how='left')

In [ ]:
# Some datapoints have NA for the joined values because of the cleaning of the weather data previously
# Hence have to drop NA values in merged dataframe
merged_df.dropna(inplace=True)

In [ ]:
merged_df.drop(['Hour'], axis=1, inplace=True)
merged_df.rename(columns={'Time (LST)': 'Hour'}, inplace=True)
merged_df = merged_df.drop(['Date/Time (LST)'], axis=1)
merged_df['Hour'] = merged_df['Hour'].str.split(':').str[0].astype(int)
merged_df.head()

,Station Id,Check-Outs,Check-Ins,Year,Month,Day,Hour,Temp (°C),Dew Point Temp (°C),Rel Hum (%),Wind Dir (10s deg),Wind Spd (km/h),Visibility (km),Stn Press (kPa),Weather
0,7000.0,0.0,1.0,2019.0,1.0,1.0,1,4.4,3.6,94.0,27.0,7.0,9.7,97.32,"Drizzle,Fog"
1,7000.0,0.0,2.0,2019.0,1.0,1.0,2,2.4,1.5,94.0,33.0,35.0,9.7,97.54,"Rain,Drizzle,Fog"
2,7000.0,2.0,0.0,2019.0,1.0,1.0,7,1.3,-1.0,84.0,33.0,27.0,24.1,99.04,Cloudy
3,7000.0,3.0,0.0,2019.0,1.0,1.0,11,-1.0,-5.0,75.0,33.0,28.0,24.1,99.77,Mostly Cloudy
4,7000.0,4.0,0.0,2019.0,1.0,1.0,15,-1.7,-7.3,66.0,34.0,25.0,24.1,100.10,Mostly Cloudy


In [ ]:
merged_df.to_csv('/content/ridership_with_weather_dataset.csv', index=False)

**3. Binarise "Weather" column of combined data into respective categories**

In [ ]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
weather_conditions = merged_df['Weather'].str.split(',')
mlb = MultiLabelBinarizer()
weather_encoded = mlb.fit_transform(weather_conditions)

weather_df = pd.DataFrame(weather_encoded, columns=mlb.classes_)
merged_df = pd.concat([merged_df.drop('Weather', axis=1), weather_df], axis=1)
merged_df.head()

,Station Id,Check-Outs,Check-Ins,Year,Month,Day,Hour,Temp (°C),Dew Point Temp (°C),Rel Hum (%),...,Moderate Snow,Mostly Cloudy,Rain,Rain Showers,Smoke,Snow,Snow Grains,Snow Pellets,Snow Showers,Thunderstorms
0,7000.0,0.0,1.0,2019.0,1.0,1.0,1,4.4,3.6,94.0,...,0,0,0,0,0,0,0,0,0,0
1,7000.0,0.0,2.0,2019.0,1.0,1.0,2,2.4,1.5,94.0,...,0,0,1,0,0,0,0,0,0,0
2,7000.0,2.0,0.0,2019.0,1.0,1.0,7,1.3,-1.0,84.0,...,0,0,0,0,0,0,0,0,0,0
3,7000.0,3.0,0.0,2019.0,1.0,1.0,11,-1.0,-5.0,75.0,...,0,1,0,0,0,0,0,0,0,0
4,7000.0,4.0,0.0,2019.0,1.0,1.0,15,-1.7,-7.3,66.0,...,0,1,0,0,0,0,0,0,0,0


**4. Aggregate binarised weather categories into more general categories to reduce dimension of data**

In [ ]:
import pandas as pd

# Aggregate rain-related features
merged_df['Rain-related'] = merged_df[['Heavy Rain', 'Heavy Rain Showers', 'Moderate Rain', 'Moderate Rain Showers', 'Rain', 'Rain Showers']].max(axis=1)

# Aggregate snow-related features
merged_df['Snow-related'] = merged_df[['Blowing Snow', 'Heavy Snow', 'Moderate Snow', 'Snow', 'Snow Grains', 'Snow Pellets', 'Snow Showers']].max(axis=1)

# Aggregate clear/cloudy conditions
merged_df['Clear/Cloudy'] = merged_df[['Clear', 'Mainly Clear', 'Cloudy', 'Mostly Cloudy']].max(axis=1)

# Aggregate obstructive conditions
merged_df['Obstructive'] = merged_df[['Drizzle', 'Fog', 'Haze']].max(axis=1)

# Aggregate freezing conditions
merged_df['Freezing'] = merged_df[['Freezing Drizzle', 'Freezing Fog', 'Freezing Rain', 'Ice Pellets']].max(axis=1)

# Aggregate extreme conditions
merged_df['Extreme'] = merged_df[['Moderate Hail', 'Thunderstorms']].max(axis=1)

# Smoke feature remains as is
merged_df['Smoke'] = merged_df['Smoke']

# Drop the original columns
merged_df.drop(columns=[
    'Heavy Rain', 'Heavy Rain Showers', 'Moderate Rain', 'Moderate Rain Showers', 'Rain', 'Rain Showers',
    'Blowing Snow', 'Heavy Snow', 'Moderate Snow', 'Snow', 'Snow Grains', 'Snow Pellets', 'Snow Showers',
    'Clear', 'Mainly Clear', 'Cloudy', 'Mostly Cloudy',
    'Drizzle', 'Fog', 'Haze',
    'Freezing Drizzle', 'Freezing Fog', 'Freezing Rain', 'Ice Pellets',
    'Moderate Hail', 'Thunderstorms'
], inplace=True)

# Verify the new aggregated columns
print(merged_df.columns)

Index(['Station Id', 'Check-Outs', 'Check-Ins', 'Year', 'Month', 'Day', 'Hour',
       'Temp (°C)', 'Dew Point Temp (°C)', 'Rel Hum (%)', 'Wind Dir (10s deg)',
       'Wind Spd (km/h)', 'Visibility (km)', 'Stn Press (kPa)', 'Smoke',
       'Rain-related', 'Snow-related', 'Clear/Cloudy', 'Obstructive',
       'Freezing', 'Extreme'],
      dtype='object')


In [ ]:
merged_df['Station Id'] = merged_df['Station Id'].astype('int32')
merged_df['Check-Outs'] = merged_df['Check-Outs'].astype('int32')
merged_df['Check-Ins'] = merged_df['Check-Ins'].astype('int32')
merged_df['Year'] = merged_df['Year'].astype('int16')
merged_df['Month'] = merged_df['Month'].astype('int8')
merged_df['Day'] = merged_df['Day'].astype('int8')
merged_df['Hour'] = merged_df['Hour'].astype('int8')
merged_df['Temp (°C)'] = merged_df['Temp (°C)'].astype('float32')
merged_df['Dew Point Temp (°C)'] = merged_df['Dew Point Temp (°C)'].astype('float32')
merged_df['Rel Hum (%)'] = merged_df['Rel Hum (%)'].astype('float32')
merged_df['Wind Dir (10s deg)'] = merged_df['Wind Dir (10s deg)'].astype('int8')
merged_df['Wind Spd (km/h)'] = merged_df['Wind Spd (km/h)'].astype('float32')
merged_df['Visibility (km)'] = merged_df['Visibility (km)'].astype('float32')
merged_df['Stn Press (kPa)'] = merged_df['Stn Press (kPa)'].astype('float32')

In [ ]:
# Convert all weather category columns to binary to reduce size

import pandas as pd

# Convert columns from the 15th onward to int8
for col in merged_df.columns[14:]:
    merged_df[col] = merged_df[col].astype('int8')

# Print the optimized dtypes
print(merged_df.dtypes)

Station Id               int32
Check-Outs               int32
Check-Ins                int32
Year                     int16
Month                     int8
Day                       int8
Hour                      int8
Temp (°C)              float32
Dew Point Temp (°C)    float32
Rel Hum (%)            float32
Wind Dir (10s deg)        int8
Wind Spd (km/h)        float32
Visibility (km)        float32
Stn Press (kPa)        float32
Smoke                     int8
Rain-related              int8
Snow-related              int8
Clear/Cloudy              int8
Obstructive               int8
Freezing                  int8
Extreme                   int8
dtype: object


In [ ]:
merged_df.to_csv('/content/drive/MyDrive/Research - Summer 2024/ridership_with_weather_dataset_preprocessed.csv', index=False)

# Data Preprocessing for 2024 Data (Test Dataset)

**1. Aggregate ridership data by check-ins and check-outs per hour**

In [ ]:
import pandas as pd
ridership_df = pd.read_csv('/content/ridership_2024_cleaned.csv')

In [ ]:
# Convert "Start Time" and "End Time" to datetime object
ridership_df['Start Time'] = pd.to_datetime(ridership_df['Start Time'])
ridership_df['End Time'] = pd.to_datetime(ridership_df['End Time'])

# Extract hour and date features from 'Start Time' and 'End Time'
ridership_df['Start Hour'] = ridership_df['Start Time'].dt.floor('H')
ridership_df['End Hour'] = ridership_df['End Time'].dt.floor('H')

In [ ]:
# Aggregate check-outs per hour per station
check_outs = ridership_df.groupby(['Start Station Id', 'Start Hour']).size().reset_index(name='Check-Outs')

# Aggregate check-ins per hour per station
check_ins = ridership_df.groupby(['End Station Id', 'End Hour']).size().reset_index(name='Check-Ins')

# Rename columns for clarity
check_outs.rename(columns={'Start Station Id': 'Station Id', 'Start Hour': 'Hour'}, inplace=True)
check_ins.rename(columns={'End Station Id': 'Station Id', 'End Hour': 'Hour'}, inplace=True)

# Merge check-outs and check-ins dataframes
aggregated_data = pd.merge(check_outs, check_ins, on=['Station Id', 'Hour'], how='outer').fillna(0)

# Sort by Station Id and Hour for better readability
aggregated_data.sort_values(by=['Station Id', 'Hour'], inplace=True)

**2. Combine aggregated ridership data with weather data**

In [ ]:
weather_df = pd.read_csv('/content/drive/MyDrive/Research - Summer 2024/weather_all_years_cleaned.csv')

In [ ]:
# Convert 'Date/Time (LST)' column to datetime
weather_df['Date/Time (LST)'] = pd.to_datetime(weather_df['Date/Time (LST)'])

# Extract the hour feature from 'time'
weather_df['Hour'] = weather_df['Date/Time (LST)'].dt.floor('H')
weather_df.head()

,Date/Time (LST),Year,Month,Day,Time (LST),Temp (°C),Dew Point Temp (°C),Rel Hum (%),Wind Dir (10s deg),Wind Spd (km/h),Visibility (km),Stn Press (kPa),Weather,Hour
0,2019-01-01 00:00:00,2019,1,1,00:00,4.5,3.7,94.0,18.0,14.0,6.4,97.29,"Drizzle,Fog",2019-01-01 00:00:00
1,2019-01-01 01:00:00,2019,1,1,01:00,4.4,3.6,94.0,27.0,7.0,9.7,97.32,"Drizzle,Fog",2019-01-01 01:00:00
2,2019-01-01 02:00:00,2019,1,1,02:00,2.4,1.5,94.0,33.0,35.0,9.7,97.54,"Rain,Drizzle,Fog",2019-01-01 02:00:00
3,2019-01-01 03:00:00,2019,1,1,03:00,2.2,1.2,93.0,31.0,29.0,19.3,97.89,"Rain,Drizzle,Fog",2019-01-01 03:00:00
4,2019-01-01 04:00:00,2019,1,1,04:00,1.9,0.8,92.0,31.0,30.0,24.1,98.16,Cloudy,2019-01-01 04:00:00


In [ ]:
# Merge the weather data with the ridership data
merged_df = pd.merge(aggregated_data, weather_df, on='Hour', how='left')

In [ ]:
# Some datapoints have NA for the joined values because of the cleaning of the weather data previously
# Hence have to drop NA values in merged dataframe
merged_df.dropna(inplace=True)

In [ ]:
merged_df.drop(['Hour'], axis=1, inplace=True)
merged_df.rename(columns={'Time (LST)': 'Hour'}, inplace=True)
merged_df = merged_df.drop(['Date/Time (LST)'], axis=1)
merged_df['Hour'] = merged_df['Hour'].str.split(':').str[0].astype(int)
merged_df.head()

,Station Id,Check-Outs,Check-Ins,Year,Month,Day,Hour,Temp (°C),Dew Point Temp (°C),Rel Hum (%),Wind Dir (10s deg),Wind Spd (km/h),Visibility (km),Stn Press (kPa),Weather
0,7000.0,1.0,0.0,2024,1,1,0,-1.5,-2.5,93.0,5.0,19.0,6.4,99.54,Snow
1,7000.0,0.0,1.0,2024,1,1,2,-1.9,-2.5,95.0,7.0,11.0,6.4,99.58,Snow
2,7000.0,0.0,1.0,2024,1,1,3,-2.1,-3.4,91.0,7.0,10.0,9.7,99.63,Snow
3,7000.0,0.0,2.0,2024,1,1,5,-2.7,-5.0,84.0,7.0,13.0,24.1,99.68,Snow
4,7000.0,1.0,0.0,2024,1,1,8,-3.6,-6.9,78.0,3.0,10.0,24.1,99.90,Cloudy


In [ ]:
merged_df.to_csv('/content/ridership_with_weather_dataset.csv', index=False)

**3. Binarise "Weather" column of combined data into respective categories**

In [ ]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
weather_conditions = merged_df['Weather'].str.split(',')
mlb = MultiLabelBinarizer()
weather_encoded = mlb.fit_transform(weather_conditions)

weather_df = pd.DataFrame(weather_encoded, columns=mlb.classes_)
merged_df = pd.concat([merged_df.drop('Weather', axis=1), weather_df], axis=1)
merged_df.head()

,Station Id,Check-Outs,Check-Ins,Year,Month,Day,Hour,Temp (°C),Dew Point Temp (°C),Rel Hum (%),...,Fog,Ice Pellets,Mainly Clear,Moderate Rain,Moderate Snow,Mostly Cloudy,Rain,Snow,Snow Grains,Snow Showers
0,7000.0,1.0,0.0,2024,1,1,0,-1.5,-2.5,93.0,...,0,0,0,0,0,0,0,1,0,0
1,7000.0,0.0,1.0,2024,1,1,2,-1.9,-2.5,95.0,...,0,0,0,0,0,0,0,1,0,0
2,7000.0,0.0,1.0,2024,1,1,3,-2.1,-3.4,91.0,...,0,0,0,0,0,0,0,1,0,0
3,7000.0,0.0,2.0,2024,1,1,5,-2.7,-5.0,84.0,...,0,0,0,0,0,0,0,1,0,0
4,7000.0,1.0,0.0,2024,1,1,8,-3.6,-6.9,78.0,...,0,0,0,0,0,0,0,0,0,0


**4. Aggregate binarised weather categories into more general categories to reduce dimension of data**

In [ ]:
import pandas as pd

# Aggregate rain-related features
merged_df['Rain-related'] = merged_df[['Moderate Rain', 'Rain']].max(axis=1)

# Aggregate snow-related features
merged_df['Snow-related'] = merged_df[['Blowing Snow', 'Moderate Snow', 'Snow', 'Snow Grains', 'Snow Showers']].max(axis=1)

# Aggregate clear/cloudy conditions
merged_df['Clear/Cloudy'] = merged_df[['Clear', 'Mainly Clear', 'Cloudy', 'Mostly Cloudy']].max(axis=1)

# Aggregate obstructive conditions
merged_df['Obstructive'] = merged_df[['Drizzle', 'Fog']].max(axis=1)

# Aggregate freezing conditions
merged_df['Freezing'] = merged_df[['Ice Pellets']].max(axis=1)

# Aggregate extreme conditions
merged_df['Extreme'] = 0

# Smoke feature remains as is
merged_df['Smoke'] = 0

# Drop the original columns
merged_df.drop(columns=[
    'Moderate Rain', 'Rain',
    'Blowing Snow', 'Moderate Snow', 'Snow', 'Snow Grains', 'Snow Showers',
    'Clear', 'Mainly Clear', 'Cloudy', 'Mostly Cloudy',
    'Drizzle', 'Fog',
    'Ice Pellets'
], inplace=True)

# Verify the new aggregated columns
print(merged_df.columns)

Index(['Station Id', 'Check-Outs', 'Check-Ins', 'Year', 'Month', 'Day', 'Hour',
       'Temp (°C)', 'Dew Point Temp (°C)', 'Rel Hum (%)', 'Wind Dir (10s deg)',
       'Wind Spd (km/h)', 'Visibility (km)', 'Stn Press (kPa)', 'Rain-related',
       'Snow-related', 'Clear/Cloudy', 'Obstructive', 'Freezing', 'Extreme',
       'Smoke'],
      dtype='object')


In [ ]:
merged_df['Station Id'] = merged_df['Station Id'].astype('int32')
merged_df['Check-Outs'] = merged_df['Check-Outs'].astype('int32')
merged_df['Check-Ins'] = merged_df['Check-Ins'].astype('int32')
merged_df['Year'] = merged_df['Year'].astype('int16')
merged_df['Month'] = merged_df['Month'].astype('int8')
merged_df['Day'] = merged_df['Day'].astype('int8')
merged_df['Hour'] = merged_df['Hour'].astype('int8')
merged_df['Temp (°C)'] = merged_df['Temp (°C)'].astype('float32')
merged_df['Dew Point Temp (°C)'] = merged_df['Dew Point Temp (°C)'].astype('float32')
merged_df['Rel Hum (%)'] = merged_df['Rel Hum (%)'].astype('float32')
merged_df['Wind Dir (10s deg)'] = merged_df['Wind Dir (10s deg)'].astype('int8')
merged_df['Wind Spd (km/h)'] = merged_df['Wind Spd (km/h)'].astype('float32')
merged_df['Visibility (km)'] = merged_df['Visibility (km)'].astype('float32')
merged_df['Stn Press (kPa)'] = merged_df['Stn Press (kPa)'].astype('float32')

In [ ]:
# Convert all weather category columns to binary to reduce size

import pandas as pd

# Convert columns from the 15th onward to int8
for col in merged_df.columns[14:]:
    merged_df[col] = merged_df[col].astype('int8')

# Print the optimized dtypes
print(merged_df.dtypes)

Station Id               int32
Check-Outs               int32
Check-Ins                int32
Year                     int16
Month                     int8
Day                       int8
Hour                      int8
Temp (°C)              float32
Dew Point Temp (°C)    float32
Rel Hum (%)            float32
Wind Dir (10s deg)        int8
Wind Spd (km/h)        float32
Visibility (km)        float32
Stn Press (kPa)        float32
Rain-related              int8
Snow-related              int8
Clear/Cloudy              int8
Obstructive               int8
Freezing                  int8
Extreme                   int8
Smoke                     int8
dtype: object


In [ ]:
# List of top 15 "Station Id" with most datapoints
list_of_stn = [7006, 7033, 7121, 7100, 7076, 7022, 7030, 7007,
              7044, 7089, 7389, 7078, 7038, 7253, 7102]

merged_df = merged_df[merged_df["Station Id"].isin(list_of_stn)]

In [ ]:
merged_df.drop(columns=['Dew Point Temp (°C)'], inplace=True)
merged_df.drop(columns=['Stn Press (kPa)', 'Wind Dir (10s deg)'], inplace=True)

merged_df.head()

<ipython-input-48-b88b9dc06ebc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df.drop(columns=['Dew Point Temp (°C)'], inplace=True)
<ipython-input-48-b88b9dc06ebc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df.drop(columns=['Stn Press (kPa)', 'Wind Dir (10s deg)'], inplace=True)


,Station Id,Check-Outs,Check-Ins,Year,Month,Day,Hour,Temp (°C),Rel Hum (%),Wind Spd (km/h),Visibility (km),Rain-related,Snow-related,Clear/Cloudy,Obstructive,Freezing,Extreme,Smoke
2098,7006,0,3,2024,1,1,1,-1.6,93.0,16.0,6.4,0,1,0,0,0,0,0
2099,7006,1,0,2024,1,1,7,-3.7,76.0,8.0,24.1,0,0,1,0,0,0,0
2100,7006,2,0,2024,1,1,10,-3.3,79.0,7.0,24.1,0,0,1,0,0,0,0
2101,7006,0,1,2024,1,1,12,-2.7,83.0,4.0,24.1,0,0,1,0,0,0,0
2102,7006,2,2,2024,1,1,14,-2.0,82.0,7.0,24.1,0,0,1,0,0,0,0


In [ ]:
merged_df.to_csv('/content/drive/MyDrive/Research - Summer 2024/ridership_2024_with_weather_dataset_preprocessed.csv', index=False)

# Generate Forecast Dataset for Prediction (March 2024)

**1. Generate Data with rows for each hour of each station in March 2024**

In [ ]:
import pandas as pd
import numpy as np

station_ids = [7006, 7033, 7121, 7100, 7076, 7022, 7030, 7007,
               7044, 7089, 7389, 7078, 7038, 7253, 7102]

# Define the date range for March
year = 2024
month = 3
days = range(1, 32)

# Generate the hourly timestamps for the day
hours = list(range(24))

data = []

# Populate the data for each station
for station_id in station_ids:
    for day in days:
        for hour in hours:
            data.append([station_id, year, month, day, hour])

ridership_df = pd.DataFrame(data, columns=["Station Id", "Year", "Month", "Day", "Hour"])

ridership_df

,Station Id,Year,Month,Day,Hour
0,7006,2024,3,1,0
1,7006,2024,3,1,1
2,7006,2024,3,1,2
3,7006,2024,3,1,3
4,7006,2024,3,1,4
...,...,...,...,...,...
11155,7102,2024,3,31,19
11156,7102,2024,3,31,20
11157,7102,2024,3,31,21
11158,7102,2024,3,31,22


**2. Combine aggregated ridership data with weather data**

In [ ]:
weather_df = pd.read_csv('/content/drive/MyDrive/Research - Summer 2024/weather_all_years_cleaned.csv')

In [ ]:
# Convert 'Date/Time (LST)' column to datetime
weather_df['Date/Time (LST)'] = pd.to_datetime(weather_df['Date/Time (LST)'])

# Extract the hour feature from 'time'
weather_df['Hour'] = weather_df['Date/Time (LST)'].dt.hour
weather_df.head()

,Date/Time (LST),Year,Month,Day,Time (LST),Temp (°C),Dew Point Temp (°C),Rel Hum (%),Wind Dir (10s deg),Wind Spd (km/h),Visibility (km),Stn Press (kPa),Weather,Hour
0,2019-01-01 00:00:00,2019,1,1,00:00,4.5,3.7,94.0,18.0,14.0,6.4,97.29,"Drizzle,Fog",0
1,2019-01-01 01:00:00,2019,1,1,01:00,4.4,3.6,94.0,27.0,7.0,9.7,97.32,"Drizzle,Fog",1
2,2019-01-01 02:00:00,2019,1,1,02:00,2.4,1.5,94.0,33.0,35.0,9.7,97.54,"Rain,Drizzle,Fog",2
3,2019-01-01 03:00:00,2019,1,1,03:00,2.2,1.2,93.0,31.0,29.0,19.3,97.89,"Rain,Drizzle,Fog",3
4,2019-01-01 04:00:00,2019,1,1,04:00,1.9,0.8,92.0,31.0,30.0,24.1,98.16,Cloudy,4


In [ ]:
# Merge the weather data with the ridership data
merged_df = pd.merge(ridership_df, weather_df, on=['Year', 'Month', 'Day', 'Hour'], how='left')

In [ ]:
# Some datapoints have NA for the joined values because of the cleaning of the weather data previously
# Hence have to drop NA values in merged dataframe
merged_df.dropna(inplace=True)

In [ ]:
merged_df

,Station Id,Year,Month,Day,Hour,Date/Time (LST),Time (LST),Temp (°C),Dew Point Temp (°C),Rel Hum (%),Wind Dir (10s deg),Wind Spd (km/h),Visibility (km),Stn Press (kPa),Weather
0,7006,2024,3,1,0,2024-03-01 00:00:00,00:00,-4.4,-10.6,62.0,25.0,10.0,24.1,100.81,Mostly Cloudy
1,7006,2024,3,1,1,2024-03-01 01:00:00,01:00,-3.9,-10.5,60.0,25.0,4.0,24.1,100.80,Mostly Cloudy
2,7006,2024,3,1,2,2024-03-01 02:00:00,02:00,-3.9,-10.5,60.0,17.0,5.0,24.1,100.81,Mostly Cloudy
3,7006,2024,3,1,3,2024-03-01 03:00:00,03:00,-3.5,-9.9,61.0,20.0,9.0,24.1,100.78,Mostly Cloudy
4,7006,2024,3,1,4,2024-03-01 04:00:00,04:00,-3.2,-9.3,63.0,21.0,7.0,24.1,100.77,Cloudy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11155,7102,2024,3,31,19,2024-03-31 19:00:00,19:00,7.5,-1.2,54.0,32.0,14.0,24.1,99.32,Mostly Cloudy
11156,7102,2024,3,31,20,2024-03-31 20:00:00,20:00,7.0,-1.4,55.0,33.0,8.0,24.1,99.34,Mostly Cloudy
11157,7102,2024,3,31,21,2024-03-31 21:00:00,21:00,6.6,-1.1,58.0,31.0,13.0,24.1,99.36,Mostly Cloudy
11158,7102,2024,3,31,22,2024-03-31 22:00:00,22:00,4.0,-2.0,65.0,34.0,19.0,24.1,99.42,Mostly Cloudy


In [ ]:
merged_df.drop(['Time (LST)'], axis=1, inplace=True)
merged_df.drop(['Date/Time (LST)'], axis=1, inplace=True)
merged_df.head()

,Station Id,Year,Month,Day,Hour,Temp (°C),Dew Point Temp (°C),Rel Hum (%),Wind Dir (10s deg),Wind Spd (km/h),Visibility (km),Stn Press (kPa),Weather
0,7006,2024,3,1,0,-4.4,-10.6,62.0,25.0,10.0,24.1,100.81,Mostly Cloudy
1,7006,2024,3,1,1,-3.9,-10.5,60.0,25.0,4.0,24.1,100.80,Mostly Cloudy
2,7006,2024,3,1,2,-3.9,-10.5,60.0,17.0,5.0,24.1,100.81,Mostly Cloudy
3,7006,2024,3,1,3,-3.5,-9.9,61.0,20.0,9.0,24.1,100.78,Mostly Cloudy
4,7006,2024,3,1,4,-3.2,-9.3,63.0,21.0,7.0,24.1,100.77,Cloudy


**3. Binarise "Weather" column of combined data into respective categories**

In [ ]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
weather_conditions = merged_df['Weather'].str.split(',')
mlb = MultiLabelBinarizer()
weather_encoded = mlb.fit_transform(weather_conditions)

weather_df = pd.DataFrame(weather_encoded, columns=mlb.classes_)
merged_df = pd.concat([merged_df.drop('Weather', axis=1), weather_df], axis=1)
merged_df.head()

,Station Id,Year,Month,Day,Hour,Temp (°C),Dew Point Temp (°C),Rel Hum (%),Wind Dir (10s deg),Wind Spd (km/h),...,Cloudy,Drizzle,Fog,Mainly Clear,Moderate Rain,Moderate Snow,Mostly Cloudy,Rain,Snow,Snow Showers
0,7006.0,2024.0,3.0,1.0,0.0,-4.4,-10.6,62.0,25.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,7006.0,2024.0,3.0,1.0,1.0,-3.9,-10.5,60.0,25.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,7006.0,2024.0,3.0,1.0,2.0,-3.9,-10.5,60.0,17.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,7006.0,2024.0,3.0,1.0,3.0,-3.5,-9.9,61.0,20.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,7006.0,2024.0,3.0,1.0,4.0,-3.2,-9.3,63.0,21.0,7.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**4. Aggregate binarised weather categories into more general categories to reduce dimension of data**

In [ ]:
import pandas as pd

# Aggregate rain-related features
merged_df['Rain-related'] = merged_df[['Moderate Rain', 'Rain']].max(axis=1)

# Aggregate snow-related features
merged_df['Snow-related'] = merged_df[['Moderate Snow', 'Snow', 'Snow Showers']].max(axis=1)

# Aggregate clear/cloudy conditions
merged_df['Clear/Cloudy'] = merged_df[['Clear', 'Mainly Clear', 'Cloudy', 'Mostly Cloudy']].max(axis=1)

# Aggregate obstructive conditions
merged_df['Obstructive'] = merged_df[['Drizzle', 'Fog']].max(axis=1)

# Aggregate freezing conditions
merged_df['Freezing'] = 0

# Aggregate extreme conditions
merged_df['Extreme'] = 0

# Smoke feature remains as is
merged_df['Smoke'] = 0

# Drop the original columns
merged_df.drop(columns=[
    'Moderate Rain', 'Rain',
    'Moderate Snow', 'Snow', 'Snow Showers',
    'Clear', 'Mainly Clear', 'Cloudy', 'Mostly Cloudy',
    'Drizzle', 'Fog'
], inplace=True)

# Verify the new aggregated columns
print(merged_df.columns)

Index(['Station Id', 'Year', 'Month', 'Day', 'Hour', 'Temp (°C)',
       'Dew Point Temp (°C)', 'Rel Hum (%)', 'Wind Dir (10s deg)',
       'Wind Spd (km/h)', 'Visibility (km)', 'Stn Press (kPa)', 'Rain-related',
       'Snow-related', 'Clear/Cloudy', 'Obstructive', 'Freezing', 'Extreme',
       'Smoke'],
      dtype='object')


In [ ]:
# show na rows
merged_df[merged_df.isna().any(axis=1)]

,Station Id,Year,Month,Day,Hour,Temp (°C),Dew Point Temp (°C),Rel Hum (%),Wind Dir (10s deg),Wind Spd (km/h),Visibility (km),Stn Press (kPa),Rain-related,Snow-related,Clear/Cloudy,Obstructive,Freezing,Extreme,Smoke
11145,7102.0,2024.0,3.0,31.0,9.0,3.2,-3.0,64.0,33.0,8.0,24.1,99.46,NaN,NaN,NaN,NaN,0,0,0
11146,7102.0,2024.0,3.0,31.0,10.0,4.3,-3.0,59.0,29.0,10.0,24.1,99.44,NaN,NaN,NaN,NaN,0,0,0
11147,7102.0,2024.0,3.0,31.0,11.0,6.1,-2.2,55.0,27.0,13.0,24.1,99.48,NaN,NaN,NaN,NaN,0,0,0
11148,7102.0,2024.0,3.0,31.0,12.0,5.9,-3.2,52.0,29.0,9.0,24.1,99.44,NaN,NaN,NaN,NaN,0,0,0
11149,7102.0,2024.0,3.0,31.0,13.0,8.1,-1.9,49.0,28.0,16.0,24.1,99.42,NaN,NaN,NaN,NaN,0,0,0
11150,7102.0,2024.0,3.0,31.0,14.0,9.6,-2.2,44.0,26.0,18.0,24.1,99.38,NaN,NaN,NaN,NaN,0,0,0
11151,7102.0,2024.0,3.0,31.0,15.0,9.9,-2.2,43.0,23.0,13.0,24.1,99.36,NaN,NaN,NaN,NaN,0,0,0
11152,7102.0,2024.0,3.0,31.0,16.0,10.4,-2.1,42.0,26.0,12.0,24.1,99.30,NaN,NaN,NaN,NaN,0,0,0
11153,7102.0,2024.0,3.0,31.0,17.0,8.6,-0.2,54.0,16.0,6.0,24.1,99.32,NaN,NaN,NaN,NaN,0,0,0
11154,7102.0,2024.0,3.0,31.0,18.0,7.9,0.4,59.0,7.0,3.0,24.1,99.34,NaN,NaN,NaN,NaN,0,0,0


In [ ]:
merged_df.dropna(inplace=True)

In [ ]:
merged_df['Station Id'] = merged_df['Station Id'].astype('int32')
merged_df['Year'] = merged_df['Year'].astype('int16')
merged_df['Month'] = merged_df['Month'].astype('int8')
merged_df['Day'] = merged_df['Day'].astype('int8')
merged_df['Hour'] = merged_df['Hour'].astype('int8')
merged_df['Temp (°C)'] = merged_df['Temp (°C)'].astype('float32')
merged_df['Dew Point Temp (°C)'] = merged_df['Dew Point Temp (°C)'].astype('float32')
merged_df['Rel Hum (%)'] = merged_df['Rel Hum (%)'].astype('float32')
merged_df['Wind Dir (10s deg)'] = merged_df['Wind Dir (10s deg)'].astype('int8')
merged_df['Wind Spd (km/h)'] = merged_df['Wind Spd (km/h)'].astype('float32')
merged_df['Visibility (km)'] = merged_df['Visibility (km)'].astype('float32')
merged_df['Stn Press (kPa)'] = merged_df['Stn Press (kPa)'].astype('float32')

In [ ]:
# Convert all weather category columns to binary to reduce size

import pandas as pd

# Convert columns from the 15th onward to int8
for col in merged_df.columns[14:]:
    merged_df[col] = merged_df[col].astype('int8')

# Print the optimized dtypes
print(merged_df.dtypes)

Station Id               int32
Year                     int16
Month                     int8
Day                       int8
Hour                      int8
Temp (°C)              float32
Dew Point Temp (°C)    float32
Rel Hum (%)            float32
Wind Dir (10s deg)        int8
Wind Spd (km/h)        float32
Visibility (km)        float32
Stn Press (kPa)        float32
Rain-related           float64
Snow-related           float64
Clear/Cloudy              int8
Obstructive               int8
Freezing                  int8
Extreme                   int8
Smoke                     int8
dtype: object


In [ ]:
merged_df.drop(columns=['Dew Point Temp (°C)'], inplace=True)
merged_df.drop(columns=['Stn Press (kPa)', 'Wind Dir (10s deg)'], inplace=True)

merged_df.head()

,Station Id,Year,Month,Day,Hour,Temp (°C),Rel Hum (%),Wind Spd (km/h),Visibility (km),Rain-related,Snow-related,Clear/Cloudy,Obstructive,Freezing,Extreme,Smoke
0,7006,2024,3,1,0,-4.4,62.0,10.0,24.1,0.0,0.0,1,0,0,0,0
1,7006,2024,3,1,1,-3.9,60.0,4.0,24.1,0.0,0.0,1,0,0,0,0
2,7006,2024,3,1,2,-3.9,60.0,5.0,24.1,0.0,0.0,1,0,0,0,0
3,7006,2024,3,1,3,-3.5,61.0,9.0,24.1,0.0,0.0,1,0,0,0,0
4,7006,2024,3,1,4,-3.2,63.0,7.0,24.1,0.0,0.0,1,0,0,0,0


In [ ]:
merged_df.to_csv('/content/drive/MyDrive/Research - Summer 2024/ridership_2024_March.csv', index=False)